### data load

In [1]:
import json

In [16]:
train_dials = json.load(open('../../input/data/train_dataset/train_dials.json'))
slot_meta = json.load(open('../../input/data/train_dataset/slot_meta.json'))
ontology = json.load(open('../../input/data/train_dataset/ontology.json'))

### train_dials

In [50]:
print("len_train_dials:", len(train_dials))
print("train_dials[0].keys(): ", train_dials[0].keys())

len_train_dials: 7000
train_dials[0].keys():  dict_keys(['dialogue_idx', 'domains', 'dialogue'])


### slot_meta

In [21]:
print(type(slot_meta))
slot_meta[:5]

<class 'list'>


['관광-경치 좋은', '관광-교육적', '관광-도보 가능', '관광-문화 예술', '관광-역사적']

### ontology

In [27]:
print(type(ontology))
print(len(ontology))
ontology.keys()

<class 'dict'>
45


dict_keys(['관광-경치 좋은', '관광-교육적', '관광-도보 가능', '관광-문화 예술', '관광-역사적', '관광-이름', '관광-종류', '관광-주차 가능', '관광-지역', '숙소-가격대', '숙소-도보 가능', '숙소-수영장 유무', '숙소-스파 유무', '숙소-예약 기간', '숙소-예약 명수', '숙소-예약 요일', '숙소-이름', '숙소-인터넷 가능', '숙소-조식 가능', '숙소-종류', '숙소-주차 가능', '숙소-지역', '숙소-헬스장 유무', '숙소-흡연 가능', '식당-가격대', '식당-도보 가능', '식당-야외석 유무', '식당-예약 명수', '식당-예약 시간', '식당-예약 요일', '식당-이름', '식당-인터넷 가능', '식당-종류', '식당-주류 판매', '식당-주차 가능', '식당-지역', '식당-흡연 가능', '지하철-도착지', '지하철-출발 시간', '지하철-출발지', '택시-도착 시간', '택시-도착지', '택시-종류', '택시-출발 시간', '택시-출발지'])

###  WoS의 Domain은 몇개이며, 어떤 Domain으로 이루어져있는지 확인해봅니다

In [31]:
domains = set([slot.split('-')[0] for slot in slot_meta])
print("Domain 갯수: ", len(domains))
print("Domain 종류: ", domains)

Domain 갯수:  5
Domain 종류:  {'관광', '택시', '지하철', '식당', '숙소'}


### WoS의 Slot은 몇개이며, Domain별로 어떤 Slot으로 이루어져있는지 확인해봅니다.

In [75]:
print("slot 갯수: ", len(slot_meta))
slot_by_domain ={}
for domain in domains:
    slot_by_domain[domain] = []
    for meta in slot_meta:
        slot = meta.split('-')
        if slot[0] == domain:
            slot_by_domain[domain].append(slot[1])

print("Domain별 slot type")
for domain in slot_by_domain.keys():
    print(f'{domain} slot 갯수: {len(slot_by_domain[domain])}')
    print(f'{domain}: {slot_by_domain[domain]}')
    print()

    

slot 갯수:  45
Domain별 slot type
관광 slot 갯수: 9
관광: ['경치 좋은', '교육적', '도보 가능', '문화 예술', '역사적', '이름', '종류', '주차 가능', '지역']

택시 slot 갯수: 5
택시: ['도착 시간', '도착지', '종류', '출발 시간', '출발지']

지하철 slot 갯수: 3
지하철: ['도착지', '출발 시간', '출발지']

식당 slot 갯수: 13
식당: ['가격대', '도보 가능', '야외석 유무', '예약 명수', '예약 시간', '예약 요일', '이름', '인터넷 가능', '종류', '주류 판매', '주차 가능', '지역', '흡연 가능']

숙소 slot 갯수: 15
숙소: ['가격대', '도보 가능', '수영장 유무', '스파 유무', '예약 기간', '예약 명수', '예약 요일', '이름', '인터넷 가능', '조식 가능', '종류', '주차 가능', '지역', '헬스장 유무', '흡연 가능']



### WoS의 Value은 몇개이며, Slot당 Value가 어떻게 이루어져있는지 확인해봅니다.

In [105]:
train_dials[0]
value_by_slot_and_domain ={}
total_value_set = set()

for domain in domains:
    value_by_slot_and_domain[domain] = {}
    for meta in slot_meta:
        slot = meta.split('-')
        if slot[0] == domain:
            value_by_slot_and_domain[domain][slot[1]] = set()


for e in train_dials:
    dialogue = e['dialogue']
    for d in dialogue:
        if len(d) > 2:  # state가 없는 turn 예외 처리
            values = d['state']
            for value in values:
                domain, slot, value = value.split('-')
                value_by_slot_and_domain[domain][slot].add(value)
                total_value_set.add(value)
print(value_by_slot_and_domain['관광'])


{'경치 좋은': {'yes'}, '교육적': {'yes'}, '도보 가능': {'yes'}, '문화 예술': {'yes'}, '역사적': {'no', 'yes', 'dontcare'}, '이름': {'여의도 공원', '낙산공원', '정동극장', '남산골 한옥 마을', '스타필드 코엑스몰', '봉은사', '국립 고궁 박물관', '서초관광정보센터', '국립 민속 박물관', '별마당 도서관', '올림픽 공원', '이화여자대학교', '남대문시장', '박물관은살아있다 인사동점', '하늘공원', '삼청동길', '롯데월드타워&몰', '조계사', '동대문 디자인 플라자', '여의도 한강공원', '인왕산', '남산 공원', '동대문 (흥인지문)', '서울 월드컵 경기장 (상암 월드컵 경기장)', '선릉과 정릉', '신세계 백화점 본점', '한국 전쟁 기념관', '중부시장', '디큐브시브', '창경궁', '삼성미술관 리움', '광화문 광장', '동대문 역사 문화 공원', '대한민국 역사 박물관', '가로수길', '서울중앙성원', '서울숲', '국립국악원', '이태원', '숭례문', '동대문 쇼핑몰', '운현궁', '삼청각', '낙산 공공 미술', '이대 패션거리', '익선동한옥거리', '북악산 서울 성곽', '충무공 이순신 동상', '명동 쇼핑 거리', '고투 몰', '창덕궁', '서울로7017', '광화문', '청와대', '잠실 야구 경기장', '이화 벽화마을', '청계천', '종묘', '서울 한양 도성', '예술의전당', '국립중앙박물관', '롯데월드 아쿠아리움', '경복궁', '국립현대미술관 서울관', '북한산 국립공원', 'IFC 몰', '서울스카이', '마장 축산물시장', '신촌', '서울 어린이 대공원', '한국의 집', '코엑스 센터', '서울 대학로', '명동 성당', '러브뮤지엄', '서울 시청', '남산 케이블카', '63시티', '타임스퀘어', '롯데백화점 본점', '명동난타극장', '문화역서울 284', '인사동', '광장시장', '석촌호수', '코엑스 

In [106]:
print("Domain별 slot type별 values")
print('전체 value 갯수: ', len(total_value_set))
for domain in value_by_slot_and_domain.keys():
    print(f'{domain} slot 갯수: {len(slot_by_domain[domain])}')
    print(f'{domain}: {slot_by_domain[domain]}')
    print()
    for slot in value_by_slot_and_domain[domain]:
        print(f'{slot} value 갯수: {len(value_by_slot_and_domain[domain][slot])}')
        print(f'{slot}: {value_by_slot_and_domain[domain][slot]}')
        print()
    print()



Domain별 slot type별 values
전체 value 갯수:  1113
관광 slot 갯수: 9
관광: ['경치 좋은', '교육적', '도보 가능', '문화 예술', '역사적', '이름', '종류', '주차 가능', '지역']

경치 좋은 value 갯수: 1
경치 좋은: {'yes'}

교육적 value 갯수: 1
교육적: {'yes'}

도보 가능 value 갯수: 1
도보 가능: {'yes'}

문화 예술 value 갯수: 1
문화 예술: {'yes'}

역사적 value 갯수: 3
역사적: {'no', 'yes', 'dontcare'}

이름 value 갯수: 101
이름: {'여의도 공원', '낙산공원', '정동극장', '남산골 한옥 마을', '스타필드 코엑스몰', '봉은사', '국립 고궁 박물관', '서초관광정보센터', '국립 민속 박물관', '별마당 도서관', '올림픽 공원', '이화여자대학교', '남대문시장', '박물관은살아있다 인사동점', '하늘공원', '삼청동길', '롯데월드타워&몰', '조계사', '동대문 디자인 플라자', '여의도 한강공원', '인왕산', '남산 공원', '동대문 (흥인지문)', '서울 월드컵 경기장 (상암 월드컵 경기장)', '선릉과 정릉', '신세계 백화점 본점', '한국 전쟁 기념관', '중부시장', '디큐브시브', '창경궁', '삼성미술관 리움', '광화문 광장', '동대문 역사 문화 공원', '대한민국 역사 박물관', '가로수길', '서울중앙성원', '서울숲', '국립국악원', '이태원', '숭례문', '동대문 쇼핑몰', '운현궁', '삼청각', '낙산 공공 미술', '이대 패션거리', '익선동한옥거리', '북악산 서울 성곽', '충무공 이순신 동상', '명동 쇼핑 거리', '고투 몰', '창덕궁', '서울로7017', '광화문', '청와대', '잠실 야구 경기장', '이화 벽화마을', '청계천', '종묘', '서울 한양 도성', '예술의전당', '국립중앙박물관', '롯데월드 아쿠아리움', '경복궁', '

### Domain별로 자주 사용되는 Bigram과 Trigram은 어떤것인지 확인해봅니다.

In [123]:
from collections import Counter

def n_gram(word, n=2):
    arr = []
    for i in range(len(word) - (n-1)):  # 2-gram이므로 문자열의 끝에서 한 글자 앞까지만 반복함
        arr.append(word[i:i+n])
    return arr

- 관광

In [125]:
for domain in value_by_slot_and_domain.keys():

    current_domain = value_by_slot_and_domain[domain]
    bigram_list = []
    trigram_list = []
    for slot_type in current_domain.keys():
        for value in current_domain[slot_type]:
            bigram_list.extend(n_gram(value, 2))
            trigram_list.extend(n_gram(value, 3))
    
    print(f'{domain} bigram 최빈수: ', Counter(bigram_list).most_common()[:5])
    print(f'{domain} trigram 최빈수: ', Counter(trigram_list).most_common()[:5])
    print()

관광 bigram 최빈수:  [('서울', 22), ('울 ', 14), ('공원', 12), ('박물', 7), ('물관', 7)]
관광 trigram 최빈수:  [('서울 ', 14), ('박물관', 7), ('yes', 6), (' 공원', 5), ('대문 ', 5)]

택시 bigram 최빈수:  [(':1', 129), (':4', 125), (':0', 97), ('1:', 94), (':2', 93)]
택시 trigram 최빈수:  [(':30', 49), (':00', 47), (':50', 45), (':15', 45), (':40', 44)]

지하철 bigram 최빈수:  [('문역', 10), ('입구', 8), ('구역', 8), ('원역', 8), ('대입', 6)]
지하철 trigram 최빈수:  [('입구역', 8), ('대입구', 6), ('대문역', 6), ('12:', 6), ('사당역', 4)]

식당 bigram 최빈수:  [(':4', 106), (':5', 105), (':1', 101), (':2', 88), (':0', 87)]
식당 trigram 최빈수:  [('17:', 32), ('16:', 31), ('18:', 30), ('19:', 30), ('12:', 29)]

숙소 bigram 최빈수:  [(' 호', 22), ('호텔', 20), ('서울', 15), (' 서', 10), ('do', 9)]
숙소 trigram 최빈수:  [(' 호텔', 17), ('don', 9), ('ont', 9), ('ntc', 9), ('tca', 9)]

